In [10]:
import numpy as np
from pymoo.model.problem import Problem
from pymoo.factory import get_sampling, get_crossover, get_mutation, get_termination
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.optimize import minimize
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.model.duplicate import DefaultDuplicateElimination
from scipy.spatial.distance import cdist

class MyProblem(Problem):
    def __init__(self):
        super().__init__(n_var=2, n_obj=1, n_constr=1, xl=0, xu=10)
    def _evaluate(self, x, out, *args, **kwargs):
        x1 = x[:,0]
        x2 = x[:,1]
        f = x1 + x2
        out["F"] = f.astype(float)
        out["G"] = x[:, 0] + x[:, 1] - 10

        
mask = ["int", "real"]

sampling = MixedVariableSampling(mask, {
    "real": get_sampling("real_random"),
    "int": get_sampling("int_random")
})

crossover = MixedVariableCrossover(mask, {
    "real": get_crossover("real_sbx", prob=1.0, eta=3.0),
    "int": get_crossover("int_sbx", prob=1.0, eta=3.0)
})

mutation = MixedVariableMutation(mask, {
    "real": get_mutation("real_pm", eta=3.0),
    "int": get_mutation("int_pm", eta=3.0)
})

problem = MyProblem()


algorithm = NSGA2(
    pop_size=20,
    mutation=mutation,
    crossover=crossover,    
    sampling=sampling,
    eliminate_duplicates=False,
)

res = minimize(
    problem,
    algorithm,
    ("n_gen", 40),
    seed=1,
    save_history=True,
    verbose=True
)


n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |      20 | -0.00000E+00 |  1.247976659 |       1 |            - |            -
    2 |      40 | -0.00000E+00 | -0.00000E+00 |       1 |  1.055546755 |        ideal
    3 |      60 | -0.00000E+00 | -0.00000E+00 |       1 |  0.00000E+00 |            f
    4 |      80 | -0.00000E+00 | -0.00000E+00 |       1 |  0.074599799 |        ideal
    5 |     100 | -0.00000E+00 | -0.00000E+00 |       1 |  0.049977920 |        ideal
    6 |     120 | -0.00000E+00 | -0.00000E+00 |       1 |  0.002695538 |        ideal
    7 |     140 | -0.00000E+00 | -0.00000E+00 |       1 |  9.05553E+24 |            f
    8 |     160 | -0.00000E+00 | -0.00000E+00 |       1 |  5.97807E+23 |            f
    9 |     180 | -0.00000E+00 | -0.00000E+00 |       1 |  4.28615E+25 |            f
   10 |     200 | -0.00000E+00 | -0.00000E+00 |       1 |  0.00000E+00 |            f
   11 |     220 | -0.00000E+00 | -0.00000E+00 |       

In [6]:
print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

Best solution found: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Function value: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Constraint violation: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


# More complex MILP

In [12]:
import numpy as np
from pymoo.model.problem import Problem
from pymoo.factory import get_sampling, get_crossover, get_mutation, get_termination
from pymoo.operators.mixed_variable_operator import MixedVariableSampling, MixedVariableMutation, MixedVariableCrossover
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.optimize import minimize
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.model.duplicate import DefaultDuplicateElimination
from scipy.spatial.distance import cdist

class MyProblem(Problem):
    def __init__(self):
        super().__init__(n_var=3, n_obj=1, n_constr=3, xl=0, xu=10)
    def _evaluate(self, x, out, *args, **kwargs):
        x1 = x[:,0]
        x2 = x[:,1]
        y1 = x[:,2]
        f = -5 * x1 - 8 * x2 - 9 * y1 
        g1 = x1 + x2 - 6 
        g2 = 5 * x1 + 9 * x2 - 45
        g3 = x1 - 1000 * y1
        out["F"] = f.astype(float)
        out["G"] = np.column_stack([g1,g2,g3])

        
mask = ["int","int", "bin"]

sampling = MixedVariableSampling(mask, {
    "bin": get_sampling("bin_random"),
    "int": get_sampling("int_random")
})

crossover = MixedVariableCrossover(mask, {
    "bin": get_crossover("bin_ux", prob=1.0),
    "int": get_crossover("int_sbx", prob=1.0, eta=3.0)
})

mutation = MixedVariableMutation(mask, {
    "bin": get_mutation("bin_bitflip"), # bin_bitflip
    "int": get_mutation("int_pm", eta=3.0)
})

problem = MyProblem()


algorithm = NSGA2(
    pop_size=20,
    mutation=mutation,
    crossover=crossover,    
    sampling=sampling,
    eliminate_duplicates=False,
)

res = minimize(
    problem,
    algorithm,
    ("n_gen", 40),
    seed=1,
    save_history=True,
    verbose=True
)


n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |      20 | -0.00000E+00 |  3.63000E+01 |       2 |            - |            -
    2 |      40 | -0.00000E+00 |  3.700000000 |       2 |  0.00000E+00 |            f
    3 |      60 |  0.00000E+00 |  0.300000000 |       1 |  2.000000000 |        ideal
    4 |      80 |  0.00000E+00 |  0.00000E+00 |       1 |  0.00000E+00 |            f
    5 |     100 |  0.00000E+00 |  0.00000E+00 |       1 |  0.00000E+00 |            f
    6 |     120 |  0.00000E+00 |  0.00000E+00 |       1 |  0.00000E+00 |            f
    7 |     140 |  0.00000E+00 |  0.00000E+00 |       1 |  0.00000E+00 |            f
    8 |     160 |  0.00000E+00 |  0.00000E+00 |       1 |  1.000000000 |        ideal
    9 |     180 |  0.00000E+00 |  0.00000E+00 |       1 |  0.00000E+00 |            f
   10 |     200 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
   11 |     220 |  0.00000E+00 |  0.00000E+00 |       

In [13]:
print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

Best solution found: [[0 5 True]
 [0 5 True]
 [0 5 True]
 [0 5 True]
 [0 5 True]
 [0 5 True]]
Function value: [[-49.]
 [-49.]
 [-49.]
 [-49.]
 [-49.]
 [-49.]]
Constraint violation: [[0.0]
 [0.0]
 [0.0]
 [0.0]
 [0.0]
 [0.0]]
